# 1. 기본 작업

In [17]:
# 라이브러리 로드
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
# from multiprocessing import Pool
# import itertools
from tqdm import tqdm
# from multiprocessing import Manager
# from concurrent.futures import ThreadPoolExecutor

In [18]:
# 데이터셋 로드
kia_video = pd.read_csv('/content/KIA_video_df.csv')  # 기아 영상
kia_comment = pd.read_csv('/content/kia_with_sentiment.csv')  # 기아 댓글
samsung_video = pd.read_csv('/content/SAMSUNG_video_df.csv')  # 삼성 영상
samsung_comment = pd.read_csv('/content/samsung_with_sentiment.csv')  # 삼성 댓글

In [19]:
kia_comment['final_sentiment'].value_counts()

,count
final_sentiment,
KIA,8754
unknown,2464
SAMSUNG,61


In [20]:
samsung_comment['final_sentiment'].value_counts()

,count
final_sentiment,
SAMSUNG,6361
unknown,1008
KIA,18


In [21]:
# 날짜 기준 데이터 분할 함수
def filter_data(video_data, comment_data, start_date, end_date):
    """
    기간 조건에 따라 유튜브 영상 및 댓글 데이터를 필터링하는 함수.

    Parameters:
        video_data (pd.DataFrame): 영상 데이터
        comment_data (pd.DataFrame): 댓글 데이터
        video_start (str): 영상 시작 일자 (YYYY-MM-DD 형식).
        video_end (str): 영상 끝 일자 (YYYY-MM-DD 형식).
        comment_start (str): 댓글 시작 일자 (YYYY-MM-DD 형식).
        comment_end (str): 댓글 끝 일자 (YYYY-MM-DD 형식).

    Returns:
        pd.DataFrame: 필터링된 댓글 데이터.
    """
    # NaN 값을 빈 문자열로 대체하고, 모든 값을 문자열로 변환한 후 날짜 추출
    video_data['publishedAt'] = video_data['publishedAt'].astype(str).str[:10]
    comment_data['PublishedAt'] = comment_data['PublishedAt'].astype(str).str[:10]

    # 문자열을 날짜 형식으로 변환
    video_data['publishedAt'] = pd.to_datetime(video_data['publishedAt'], errors='coerce')
    comment_data['PublishedAt'] = pd.to_datetime(comment_data['PublishedAt'], errors='coerce')

    # 영상 필터링
    filtered_videos = video_data[
        (video_data['publishedAt'] >= pd.to_datetime(start_date)) &
        (video_data['publishedAt'] <= pd.to_datetime(end_date))
    ]

    # 댓글 필터링 (필터링된 영상에 해당하는 댓글만 포함)
    filtered_comments = comment_data[
        (comment_data['VideoID'].isin(filtered_videos['videoId'])) &
        (comment_data['PublishedAt'] >= pd.to_datetime(start_date)) &
        (comment_data['PublishedAt'] <= pd.to_datetime(end_date))
    ]

    return filtered_comments

In [22]:
# 기아 데이터 분할 (한국시리즈 종료 전)
kia_replies_1019_1021 = filter_data(kia_video, kia_comment, "2024-10-19", "2024-10-21")
kia_replies_1019_1023 = filter_data(kia_video, kia_comment, "2024-10-19", "2024-10-23")
kia_replies_1019_1025 = filter_data(kia_video, kia_comment, "2024-10-19", "2024-10-25")
kia_replies_1019_1026 = filter_data(kia_video, kia_comment, "2024-10-19", "2024-10-26")
kia_replies_1019_1028 = filter_data(kia_video, kia_comment, "2024-10-19", "2024-10-28")

# 기아 데이터 분할 (한국시리즈 종료 후)
kia_replies_1029_1031 = filter_data(kia_video, kia_comment, "2024-10-29", "2024-10-31")
kia_replies_1029_1101 = filter_data(kia_video, kia_comment, "2024-10-29", "2024-11-01")
kia_replies_1029_1102 = filter_data(kia_video, kia_comment, "2024-10-29", "2024-11-02")
kia_replies_1029_1103 = filter_data(kia_video, kia_comment, "2024-10-29", "2024-11-03")
kia_replies_1029_1104 = filter_data(kia_video, kia_comment, "2024-10-29", "2024-11-04")

In [23]:
# 삼성 데이터 분할 (한국시리즈 종료 전)
samsung_replies_1019_1021 = filter_data(samsung_video, samsung_comment, "2024-10-19", "2024-10-21")
samsung_replies_1019_1023 = filter_data(samsung_video, samsung_comment, "2024-10-19", "2024-10-23")
samsung_replies_1019_1025 = filter_data(samsung_video, samsung_comment, "2024-10-19", "2024-10-25")
samsung_replies_1019_1026 = filter_data(samsung_video, samsung_comment, "2024-10-19", "2024-10-26")
samsung_replies_1019_1028 = filter_data(samsung_video, samsung_comment, "2024-10-19", "2024-10-28")

# 삼성 데이터 분할 (한국시리즈 종료 후)
samsung_replies_1029_1031 = filter_data(samsung_video, samsung_comment, "2024-10-29", "2024-10-31")
samsung_replies_1029_1101 = filter_data(samsung_video, samsung_comment, "2024-10-29", "2024-11-01")
samsung_replies_1029_1102 = filter_data(samsung_video, samsung_comment, "2024-10-29", "2024-11-02")
samsung_replies_1029_1103 = filter_data(samsung_video, samsung_comment, "2024-10-29", "2024-11-03")
samsung_replies_1029_1104 = filter_data(samsung_video, samsung_comment, "2024-10-29", "2024-11-04")

# 2. 동질성/이질성 분석

In [24]:
import itertools
import networkx as nx
import numpy as np
import pandas as pd

# 동질성 계산 함수 (밀도 고려)
def calculate_homophily_with_density(graph):
    kia_edges, samsung_edges, kia_samsung_edges = 0, 0, 0
    kia_valid_edges, samsung_valid_edges = 0, 0
    total_possible_edges = 0

    # 노드 그룹 분리
    kia_nodes = [n for n, attr in graph.nodes(data=True) if attr.get('sentiment') == 'KIA']
    samsung_nodes = [n for n, attr in graph.nodes(data=True) if attr.get('sentiment') == 'SAMSUNG']

    # 가능한 최대 엣지 수 계산
    kia_possible_edges = len(kia_nodes) * (len(kia_nodes) - 1) / 2
    samsung_possible_edges = len(samsung_nodes) * (len(samsung_nodes) - 1) / 2
    total_possible_edges = graph.number_of_nodes() * (graph.number_of_nodes() - 1) / 2

    for u, v in graph.edges:
        sentiment_u = graph.nodes[u].get('sentiment')
        sentiment_v = graph.nodes[v].get('sentiment')

        if sentiment_u in ['KIA', 'SAMSUNG'] and sentiment_v in ['KIA', 'SAMSUNG']:
            if sentiment_u == sentiment_v == 'KIA':
                kia_edges += 1
            elif sentiment_u == sentiment_v == 'SAMSUNG':
                samsung_edges += 1
            elif {sentiment_u, sentiment_v} == {'KIA', 'SAMSUNG'}:
                kia_samsung_edges += 1

    # 밀도 기반 동질성 계산
    kia_homophily = (kia_edges / kia_possible_edges) if kia_possible_edges > 0 else 0
    samsung_homophily = (samsung_edges / samsung_possible_edges) if samsung_possible_edges > 0 else 0
    kia_samsung_heterophily = (kia_samsung_edges / total_possible_edges) if total_possible_edges > 0 else 0

    return {
        'kia_homophily': kia_homophily,
        'samsung_homophily': samsung_homophily,
        'kia_samsung_heterophily': kia_samsung_heterophily
    }

# 댓글 작성자들만 노드로 하고, 같은 영상에 댓글을 남긴 작성자들끼리 엣지를 추가하는 네트워크 생성 함수
def build_author_network(kia_data, samsung_data):
    """
    댓글 작성자들만 노드로 하고, 같은 영상에 댓글을 남긴 작성자들끼리 엣지를 추가하는 네트워크를 생성합니다.
    """
    G = nx.Graph()

    # KIA와 SAMSUNG 데이터를 병합
    all_data = pd.concat([kia_data[['VideoID', 'Author', 'final_sentiment']].dropna(),
                          samsung_data[['VideoID', 'Author', 'final_sentiment']].dropna()])

    # 작성자 sentiment 정보를 딕셔너리로 정리
    author_sentiments = all_data.drop_duplicates('Author').set_index('Author')['final_sentiment'].to_dict()

    # 각 영상별 댓글 작성자 리스트 만들기
    video_groups = all_data.groupby('VideoID')['Author'].apply(list)

    # 각 영상의 작성자들 간에 엣지 추가
    for authors in video_groups:
        G.add_edges_from(itertools.combinations(authors, 2))

    # 노드 속성으로 sentiment 추가
    nx.set_node_attributes(G, author_sentiments, 'sentiment')

    return G

## 2024.10.19 - 2024.10.28

In [33]:
author_network_1019_1028 = return_author_network(kia_replies_1019_1028, samsung_replies_1019_1028)
z_1019_1028 = calculate_homophily_with_density(author_network_1019_1028)

In [34]:
z_1019_1028

{'kia_homophily': 0.16287127868249654,
 'samsung_homophily': 0.3037599291259366,
 'kia_samsung_heterophily': 0.0005783524260935825}

## 2024.10.29 - 2024.11.04

In [43]:
author_network_1029_1104 = return_author_network(kia_replies_1029_1104, samsung_replies_1029_1104)
z_1029_1104 = calculate_homophily_with_density(author_network_1029_1104)

In [44]:
z_1029_1104

{'kia_homophily': 0.28516032310313916,
 'samsung_homophily': 0.4697539927625764,
 'kia_samsung_heterophily': 0.00037779872736452884}